### Here,s simple Convolutional neural Network (CNN) on the same toy data set is implemented. This model consists of single convolutional layer+maxpool and fully connected layer. Furthermore, the concpet of dropout for regularization is also taken into account.

In [6]:
import tensorflow as tf
from util.data import *
import numpy as np
from sklearn.metrics import confusion_matrix
import sklearn.metrics

#### Data and Parameters --------------------------------------------------------------------

In [7]:
Sample_number=10000
Feature_Number=1024
batch_size=256
learning_rate=0.01
display_step=20
epochs=100

keep_conv=0.8 # uses for dropout in conv layer

DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)

Train_x = np.array(Train_x).reshape(-1, 32, 32, 1)  # input train data in the form of 32x32x1 matrix
Test_x = np.array(Test_x).reshape(-1, 32, 32, 1)  # input  test data in the form of 32x32x1 matrix
print(Train_x.shape)
print(Train_y.shape)

(6000, 32, 32, 1)
(6000, 10)


#### Structure definition -----------------------------------------------------------------

In [8]:
X=tf.placeholder(dtype=tf.float32,shape=[None,32,32,1])
Y=tf.placeholder(dtype=tf.float32,shape=[None,10])

# We are using same padding for conv layer, so the out put of each filter would be a matrix of same shape as input 32*32*1
WC=tf.Variable(tf.random_normal([3,3,1,64],stddev=0.01)) # 64 conv filters in the form of 3*3*1 (last 1 shows we use only one channel)
bC=tf.Variable(tf.random_normal([64], stddev=0.01)) 
# After applying maxpool on the output of conv_layer,since we use 2 for stride steps, the output of maxpool layer would divide the number of its input 
WF=tf.Variable(tf.random_normal([16*16*64,10],stddev=0.01)) # 64 conv filters in the form of 3*3*1 (last 1 shows we use only one channel)
bF=tf.Variable(tf.random_normal([10], stddev=0.01))

keep_conv= tf.placeholder("float")

#### Model and Graph Definition --------------------------------------------

In [12]:
def model(X,Conv_layer_W,Fully_layyer_W,Conv_bias,Fully_bias,keep_conv):
    
    conv=tf.nn.conv2d(X,Conv_layer_W,strides=[1,1,1,1],padding='SAME')
    conv=tf.nn.bias_add(conv,bC)
    conv=tf.nn.relu(conv)
    pool=tf.nn.max_pool(conv,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    pool=tf.nn.dropout(pool,keep_conv)
    # below part is necessary to reshape the output of the pooling layer suitable for matrix multiplication in fully connected layer
    Fc=tf.reshape(pool, [-1, Fully_layyer_W.get_shape().as_list()[0]])
    out=tf.add(tf.matmul(Fc,Fully_layyer_W),bF)
    
    return tf.nn.softmax(out)

Pred_prob=model(X,WC,WF,bC,bF,keep_conv)
predicted_label=tf.equal(tf.argmax(Pred_prob,1),tf.argmax(Y,1))

pred_y=tf.argmax(Pred_prob,1)

true_y=tf.argmax(Y,1)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Pred_prob,labels=Y))
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

Accuracy=tf.reduce_mean(tf.cast(predicted_label,'float'))

Precision=sklearn.metrics.precision_score(true_y, pred_y)
Recall=sklearn.metrics.recall_score(true_y, pred_y)
Fscore=sklearn.metrics.f1_score(true_y, pred_y)




ValueError: Expected array-like (array or non-string sequence), got <tf.Tensor 'ArgMax_15:0' shape=(?,) dtype=int64>

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        avg_cost=0
        for j in range(Number_of_batches):
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            sess.run(train_step,feed_dict={X:Trx,Y:Try,keep_conv:0.8})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Try,keep_conv:1})/Number_of_batches
        if i% display_step==0:
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Train_x,Y:Train_y,keep_conv:1})
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y,keep_conv:1})
            precision,recall,fscore=sess.run([Precision,Recall,Fscore],feed_dict={X:Test_x,Y:Test_y,keep_conv:1})
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (i, epochs, avg_cost, Train_Accuracy, Test_Accuracy))
            
            print ("Epoch: %03d/%03d precision: %.9f recall: %.3f fscore: %.3f" 
                   % (i, epochs, precision, recall, fscore))
sess.close()